In [82]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [83]:
trainImages = open(os.getcwd() + '/data/MINST/train-images.idx3-ubyte', 'rb')
trainLabels = open(os.getcwd() + '/data/MINST/train-labels.idx1-ubyte', 'rb')
testImages = open(os.getcwd() + '/data/MINST/t10k-images.idx3-ubyte', 'rb')
testLabels = open(os.getcwd() + '/data/MINST/t10k-labels.idx1-ubyte', 'rb')

In [84]:
assert(int.from_bytes(trainImages.read(4), "big") == 2051)
assert(int.from_bytes(trainLabels.read(4), 'big') == 2049)
assert(int.from_bytes(testImages.read(4), 'big') == 2051)
assert(int.from_bytes(testLabels.read(4), 'big') == 2049)

In [85]:
trainImagesNum = int.from_bytes(trainImages.read(4), 'big')
trainLabelsNum = int.from_bytes(trainLabels.read(4), 'big')
testLabelsNum = int.from_bytes(testLabels.read(4), 'big')
testImagesNum = int.from_bytes(testImages.read(4), 'big')

In [86]:
assert(trainImagesNum == trainLabelsNum)
assert(testImagesNum == testLabelsNum)

In [87]:
trainRows = int.from_bytes(trainImages.read(4), 'big')
trainCols = int.from_bytes(trainImages.read(4), 'big')
testRows = int.from_bytes(testImages.read(4), 'big')
testCols = int.from_bytes(testImages.read(4), 'big')

In [88]:
trainPixels = trainCols * trainRows
testPixels = testCols * testRows

In [89]:
trainX = np.zeros((trainImagesNum, trainPixels))
trainY = np.zeros(trainLabelsNum)
testX = np.zeros((testImagesNum, testPixels))
testY = np.zeros((testLabelsNum))

In [90]:
for i in range(trainImagesNum):
    for j in range(trainPixels):
        trainX[i,j] = float(int.from_bytes(trainImages.read(1), 'big'))
    trainY[i] = int.from_bytes(trainLabels.read(1), 'big')

In [91]:
for i in range(testImagesNum):
    for j in range(testPixels):
        testX[i,j] = float(int.from_bytes(testImages.read(1), 'big'))
    testY[i] = int.from_bytes(testLabels.read(1), 'big')

In [92]:
trainX = trainX/255
testX = testX/255

In [93]:
trainX = tf.convert_to_tensor(trainX)
trainY = tf.keras.utils.to_categorical(trainY, num_classes=10)
testX = tf.convert_to_tensor(testX)
testY = tf.keras.utils.to_categorical(testY, num_classes=10)

In [94]:
def simpleModel(input_shape):
    XInput = tf.keras.Input(shape=input_shape)
    X = tf.keras.layers.Dense(200, activation='relu', name='fc0')(XInput)
    X = tf.keras.layers.Dropout(0.2)(X)
    X = tf.keras.layers.Dense(200, activation='relu', name='fc1')(X)
    X = tf.keras.layers.Dropout(0.2)(X)
    X = tf.keras.layers.Dense(10,activation='softmax', name ='out')(X)
    model = tf.keras.Model(inputs=XInput, outputs = X, name = 'SimpleModel')
    return model

In [95]:
model = simpleModel(trainX.shape[1:])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

In [96]:
preds = model.fit(x=trainX, y=trainY, batch_size= 100, epochs=50, verbose=1, sample_weight=None)

Epoch 1/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0541 - accuracy: 0.9027
Epoch 2/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0245 - accuracy: 0.9578
Epoch 3/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0185 - accuracy: 0.9686
Epoch 4/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0152 - accuracy: 0.9741
Epoch 5/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0127 - accuracy: 0.9786
Epoch 6/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0110 - accuracy: 0.9808
Epoch 7/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0097 - accuracy: 0.9827
Epoch 8/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0086 - accuracy: 0.9848
Epoch 9/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0085 - accuracy: 0.9854
Epoch 10/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0071 - accuracy: 0.9876

In [97]:
preds = model.evaluate(testX, testY, batch_size=100, verbose=1, sample_weight=None)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

100/100 [==============================] - 0s 1ms/step - loss: 0.0139 - accuracy: 0.9843

Loss = 0.01391863264143467
Test Accuracy = 0.9843000173568726


In [98]:
def convModel(input_shape):
    XIn = tf.keras.Input(shape=input_shape)
    X = tf.keras.layers.Convolution2D(5, 3, data_format='channels_last',input_shape=input_shape, padding='same', activation='relu')(XIn)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)(X)
    Y = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Convolution2D(10, 3, padding='valid', activation='relu')(Y)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)(X)
    X = tf.keras.layers.BatchNormalization()(X)
    Y = tf.keras.layers.Convolution2D(10, 2)(Y)
    Y = tf.keras.layers.MaxPool2D(pool_size=(2,2))(Y)
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dropout(0.2)(X)
    X = tf.keras.layers.Dense(100, activation='relu')(X)
    X = tf.keras.layers.Dense(10, activation='softmax')(X)
    model = tf.keras.Model(inputs=XIn, outputs=X)
    return model

In [99]:
trainX = tf.reshape(trainX, shape=(trainX.shape[0], 28,28,1))
testX = tf.reshape(testX, shape=(testX.shape[0], 28,28,1))

In [100]:
model2 = convModel(trainX.shape[1:])
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [101]:
model2.fit(x=trainX, y=trainY, batch_size= 100, epochs=50, verbose=1, sample_weight=None)

Epoch 1/50
600/600 [==============================] - 2s 3ms/step - loss: 0.2584 - accuracy: 0.9189
Epoch 2/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0904 - accuracy: 0.9711
Epoch 3/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0681 - accuracy: 0.9780
Epoch 4/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0558 - accuracy: 0.9826
Epoch 5/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0451 - accuracy: 0.9856
Epoch 6/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0415 - accuracy: 0.9866
Epoch 7/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0362 - accuracy: 0.9879
Epoch 8/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0326 - accuracy: 0.9892
Epoch 9/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0312 - accuracy: 0.9894
Epoch 10/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0266 - accuracy: 0.9914

In [102]:
preds = model2.evaluate(testX, testY, batch_size=100, verbose=1, sample_weight=None)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

100/100 [==============================] - 0s 2ms/step - loss: 0.0459 - accuracy: 0.9901

Loss = 0.04587492346763611
Test Accuracy = 0.9901000261306763
